In [8]:
# Terraform an RDS instance: rds.tf
# Output the RDS connection props to a config file
# SQL script to make tables in RDS instance
# Python script to pull data, write to file, and copy to RDS instance

In [9]:
import requests

In [10]:
API_KEY = 'DotDawmB1KyneOh1BjXbGws1MVuUDS3wstyqvJ9b'
BASE_URL = 'https://api.open.fec.gov/v1'
DEFAULT_PARAMS = {}
ENDPOINTS = {
  'candidates': 'candidates'
}
session = requests.Session()

In [ ]:
def get_params(params):
  defaults = DEFAULT_PARAMS.copy()
  defaults.update(params)
  return defaults

In [31]:
def get_url(base_url, endpoint, api_key, **kwargs):
  # Pass kwargs to add params to the URL for filters, etc.
  # See params for your endpoint in the docs: https://api.open.fec.gov/developers/
  url_params = get_url_params(kwargs.copy())
  endpoint = '/'.join([base_url, endpoint])
  query_url = endpoint + '?api_key={}&'.format(api_key) + '&'.join(url_params)
  return query_url

In [28]:
def get_data(endpoint, **kwargs):
  endpoint = ENDPOINTS[endpoint]
  url = get_url(BASE_URL, endpoint, API_KEY, **kwargs)
  response = session.get(url)
  return response.json()

In [38]:
# Example usage:
data = get_data('candidates', party='DEM', year=2010, candidate_id='H6NJ05155')
page_candidates = len(data['results'])
pages = data['pagination']['pages']
candidate_id = data['results'][0]['candidate_id']

print(pages)
print(candidate_id)
print(page_candidates)

1
H6NJ05155
1


In [ ]:
# Format the data
def format_record(record):
  pass

# Write data to file
def write_to_file(record):
  pass

# Python script to read in the rds config file and run copy command 
def copy_to_rds(file):
  pass